In [33]:
-(1/3)*math.log(1/3, 2) - (1/2)*math.log(1/2, 2)

1.0283208335737188

In [34]:
-((2/5)*1 + (3/5)*1.02)

-1.012

In [1]:
from collections import Counter
import math
import pandas as pd
df = pd.read_csv("playTennis.csv")
print(len(df))
df

14


,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [3]:
def entropy(ls):
    counts = Counter(x for x in ls)
    total = len(ls)
    probs = [x/total for x in counts.values()]
    E = sum(-p * math.log(p,2) for p in probs)
    
    return E

In [6]:
#check
entropy(df['PlayTennis'])

0.9402859586706309

In [7]:
#check
dfs = df.groupby('Outlook')
for k,v in dfs:
    print(k)
    print(v)

Overcast
     Outlook Temperature Humidity    Wind PlayTennis
2   Overcast         Hot     High    Weak        Yes
6   Overcast        Cool   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
Rain
   Outlook Temperature Humidity    Wind PlayTennis
3     Rain        Mild     High    Weak        Yes
4     Rain        Cool   Normal    Weak        Yes
5     Rain        Cool   Normal  Strong         No
9     Rain        Mild   Normal    Weak        Yes
13    Rain        Mild     High  Strong         No
Sunny
   Outlook Temperature Humidity    Wind PlayTennis
0    Sunny         Hot     High    Weak         No
1    Sunny         Hot     High  Strong         No
7    Sunny        Mild     High    Weak         No
8    Sunny        Cool   Normal    Weak        Yes
10   Sunny        Mild   Normal  Strong        Yes


In [9]:
#check
dfs.agg('sum')['PlayTennis']

Outlook
Overcast     YesYesYesYes
Rain        YesYesNoYesNo
Sunny        NoNoNoYesYes
Name: PlayTennis, dtype: object

In [19]:
def inf_gain(data, a, target):
    df_split = data.groupby(a)
    n = len(data)
    df_agg = df_split.agg({target:[entropy, lambda x : len(x)/n]})[target]
    df_agg.columns = ['Entropy', 'Proportion']
#     print(df_agg)
    new_E = sum(df_agg['Entropy'] * df_agg['Proportion'])
    old_E = entropy(df[target])
    
    return old_E - new_E
    

In [20]:
#check
print(inf_gain(df, 'Outlook', 'PlayTennis'))
# inf_gain(df, 'Temperature', 'PlayTennis')

0.2467498197744391


In [27]:
def id3(df, target, attr, def_class = None, def_attr = 'S'):
    pn = Counter(x for x in df[target])
    print(f'\n ** {pn} **')
    
    if(len(pn)==1):
        return next(iter(pn))
    elif df.empty or (not attr):
        return def_class
    else:
        def_class = max(pn, key=pn.get)
        print(f'Default class {def_class}')
        
        gains = {}
        for a in attr:
            gains[a] = inf_gain(df, a , target)
            print(f'Inf gain on {a}: {gains[a]}')
            
        best = max(gains, key=gains.get)
        print(f'Max gain: {best}')
        
        tree = {best: {}} #initialise tree with best as root
        attr.remove(best)
        
        for av, data in df.groupby(best):
            subtree = id3(data, target, attr, def_class, best)
            tree[best][av] = subtree
            print(f'best {best} {av}')
            print(tree)
            
    return tree

In [28]:
attr = list(df.columns)
attr.remove('PlayTennis')
print("Predicting attributes", attr)

tree = id3(df, 'PlayTennis', attr)

Predicting attributes ['Outlook', 'Temperature', 'Humidity', 'Wind']

 ** Counter({'Yes': 9, 'No': 5}) **
Default class Yes
Inf gain on Outlook: 0.2467498197744391
Inf gain on Temperature: 0.029222565658954647
Inf gain on Humidity: 0.15183550136234136
Inf gain on Wind: 0.04812703040826927
Max gain: Outlook

 ** Counter({'Yes': 4}) **
best Outlook Overcast
{'Outlook': {'Overcast': 'Yes'}}

 ** Counter({'Yes': 3, 'No': 2}) **
Default class Yes
Inf gain on Temperature: -0.010691541762062773
Inf gain on Humidity: -0.010691541762062773
Inf gain on Wind: 0.9402859586706309
Max gain: Wind

 ** Counter({'No': 2}) **
best Wind Strong
{'Wind': {'Strong': 'No'}}

 ** Counter({'Yes': 3}) **
best Wind Weak
{'Wind': {'Strong': 'No', 'Weak': 'Yes'}}
best Outlook Rain
{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}}}

 ** Counter({'No': 3, 'Yes': 2}) **
Default class No
Inf gain on Temperature: 0.5402859586706309
Inf gain on Humidity: 0.9402859586706309
Max gain: Humi

In [29]:
#check
c = Counter(x for x in df['PlayTennis'])
print(c)
next(iter(c))

Counter({'Yes': 9, 'No': 5})


'No'

In [32]:
from pprint import pprint
print("Tree: \n")
pprint(tree)
ba = next(iter(tree))
print("Best Attribute", ba)
print("Tree Keys: \n", tree[ba].keys())

Tree: 

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
Best Attribute Outlook
Tree Keys: 
 dict_keys(['Overcast', 'Rain', 'Sunny'])
